<a href="https://colab.research.google.com/github/Luj2000/projects/blob/main/_LinearRegression_Boston_Iterative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import software libraries and load the dataset #

In [ ]:
import sys                                             # Read system parameters.
import numpy as np                                     # Work with multi-dimensional arrays and matrices.
import pandas as pd                                    # Manipulate and analyze data.
import sklearn                                         # Perform data mining and analysis.
from sklearn import datasets
from time import time                                  # Calculate training time.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- scikit-learn {}\n'.format(sklearn.__version__))

# Load the dataset.
boston = datasets.load_boston()
print('Loaded {} records.'.format(len(boston.data)))

# Convert array to pandas DataFrame.
data_raw = pd.DataFrame(boston['data'], columns = boston['feature_names'])
data_raw['target'] = boston['target']

Libraries used in this project:
- Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
- NumPy 1.21.6
- pandas 1.3.5
- scikit-learn 1.0.2

Loaded 506 records.


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

# Split the datasets

In [ ]:
from sklearn.model_selection import train_test_split

# 'target' is the dependent variable (value to be predicted), so it will be
# removed from the training data and put into a separate DataFrame for labels.
label_columns = ['target']

# Split the training and test datasets and their labels.
X_train, X_test, y_train, y_test = train_test_split(data_raw.loc[:, 'CRIM': 'LSTAT'],
                                                                            data_raw[label_columns],
                                                                            random_state = 2)

# Compare the number of rows and columns in the original data to the training and test sets.
print(f'Original set:        {data_raw.shape}')
print('------------------------------')
print(f'Training features:   {X_train.shape}')
print(f'Test features:       {X_test.shape}')
print(f'Training labels:     {y_train.shape}')
print(f'Test labels:         {y_test.shape}')

Original set:        (506, 14)
------------------------------
Training features:   (379, 13)
Test features:       (127, 13)
Training labels:     (379, 1)
Test labels:         (127, 1)


# Drop columns that won't be used for training

In [ ]:
# Drop column from dataset that shows weak correlation.
def drop_unused(dataset):
    
    print('Columns before drop:\n\n{}\n'.format(list(dataset.columns)))
        
    dataset = dataset.drop(['CHAS'], axis = 1)
    
    print('Columns after drop:\n\n{}\n'.format(list(dataset.columns)))
    return dataset

X_train, X_test = drop_unused(X_train.copy()), drop_unused(X_test.copy())

Columns before drop:

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

Columns after drop:

['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

Columns before drop:

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

Columns after drop:

['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']



# Standardize the features

In [ ]:
def standardize(X):
    result = X.copy()
    
    for feature in X.columns:
        result[feature] = (X[feature] - X[feature].mean()) / X[feature].std()  # z-score formula.
 m       
    return result

X_train = standardize(X_train)
X_test = standardize(X_test)

print('The features have been standardized.')

The features have been standardized.


# Train a model and calculate its cost

In [ ]:
from sklearn.metrics import mean_squared_error as mse

def model_train(model):
    start = time()
    model.fit(X_train, np.ravel(y_train))
    end = time()
    train_time = (end - start) * 1000
    
    predict = model.predict(X_test)
    
    cost = mse(y_test, predict)
    
    print('Linear regression model took {:.2f} milliseconds to fit.'.format(train_time))
    print('Cost (mean squared error): {:.2f}'.format(cost))
    
print('The function to train the model and calculate its cost has been defined.')

The function to train the model and calculate its cost has been defined.


# Evaluate linear regression models using both closed-form and iterative solutions

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor

# Create closed-form and iterative ridge regression models.
def model_eval(eta):
    for name, model in [
        ('Ridge regression (closed form)', Ridge(alpha = 0.1, solver = 'cholesky')),
        ('Ridge regression (gradient descent)', SGDRegressor(penalty = 'l2',
                                                             alpha = 0.1,
                                                             tol = 1e-3,
                                                             learning_rate = 'constant',
                                                             eta0 = eta,
                                                             random_state = 2))]:

        print('Model: {}'.format(name))
        print('--------------------')
        model_train(model)
        print('\n')
        
print('The function to evaluate the linear regression models has been defined.')

The function to evaluate the linear regression models has been defined.


In [ ]:
model_eval(0.09)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 3.51 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 7.28 milliseconds to fit.
Cost (mean squared error): 65.36




In [ ]:
model_eval(0.08)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 8.64 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 2.29 milliseconds to fit.
Cost (mean squared error): 47.05




In [ ]:
model_eval(0.05)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 2.32 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 5.85 milliseconds to fit.
Cost (mean squared error): 31.89




In [ ]:
model_eval(0.01)

Model: Ridge regression (closed form)
--------------------
Linear regression model took 3.23 milliseconds to fit.
Cost (mean squared error): 21.88


Model: Ridge regression (gradient descent)
--------------------
Linear regression model took 2.98 milliseconds to fit.
Cost (mean squared error): 25.25


